In [1]:
# create card with suit, rank, value
import random
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11}

In [2]:
class Cards():
    
    def __init__ (self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        
    def __str__ (self):
        return f'{self.rank} of {self.suit}'
    
    def __repr__ (self):
        return f'{self.rank} of {self.suit}'
    
    #declare __eq__ so that when comparing equality of the object, this method will tell python what attributes to compare. 
    #In this case, we are telling python to compare the value.
    #so if you compare Ace of spades vs Ace of Clubs, it will return True cause we are comparing the values
    def __eq__(self, other):
        return self.value == other.value

In [3]:
class Deck():
    
    def __init__ (self):
        self.all_cards = []
        
        for suit in suits:
            for rank in ranks:
                self.all_cards.append(Cards(suit,rank))
                
    def shuffle(self):
        random.shuffle(self.all_cards)
        
    def deal_one(self):
        return self.all_cards.pop(0)
        
    def __str__(self):
        return f'You have {len(self.all_cards)} cards in your hands'

In [78]:
class Player():
    
    def __init__(self,name):
        self.name = name
        self.wallet = 100
        self.games_played = 0
        self.games_won = 0
        self.amount_won= 0
        
    def update_wallet(self, amount, win_or_lose):
        self.wallet += amount
        self.games_played += 1
        if win_or_lose == "win":
            self.games_won += 1
        self.amount_won += amount
        
        if self.wallet < 0:
            return f"{self.name}, You are busted!"
        else:
            return f"{self.name}, You have ${self.wallet} left in your wallet"        
    
    def bet(self):
        while True:
            try:
                amount = int(input(f"{self.name}, please place your bet. You have ${self.wallet} in your wallet "))
            except ValueError:
                print("Not a number! Please try again.")
                continue
            else:
                if amount <= self.wallet:
                    print(f"{self.name}, you have bet ${amount}.")
                    return amount
                    break
                else:
                    print(f"{self.name}, You only have ${self.wallet} left in your wallet")
    
    def display_stats(self):
        print(f"{self.name}, here are your stats \n"
              f"Games played: {self.games_played} \n"
              f"Games Won: {self.games_won} \n"
              f"Winning Percentage: {'0' if self.games_played == 0 else self.games_won/self.games_played*100}%\n"
              f"Amount Won/Lost: ${self.amount_won}\n"
              f"Wallet Balance: ${self.wallet}")

In [79]:
def calculate_blackjack_total(person_hands):

    person_hands_total = 0
    person_hands_aces = 0

    for card in person_hands:
        person_hands_total += card.value

        if card.value == 11:
            person_hands_aces += 1

    while person_hands_aces != 0:
        if person_hands_total > 21:
            person_hands_total -= 10
        person_hands_aces -= 1
        
    return person_hands_total

In [80]:
## Start Game
player_name = input("Player, please input your name\n")
player1 = Player(player_name)


Player, please input your name
John


In [154]:
## Gameplay

def play_blackjack():
    no_of_rounds = 1
    continue_game = "y"
    
    while continue_game in {"y", "Y", "Yes"}:
        print(f"\n***Start Round {no_of_rounds}***")
        #Initialize

        player_hands = []
        player_hands_total = 0
        player_bet = 0
        dealer_hands = []
        dealer_hands_total = 0

        new_deck = Deck()
        new_deck.shuffle()

        #Player's turn

        player_bet = player1.bet()

        for x in range(2):
            player_hands.append(new_deck.deal_one())
            dealer_hands.append(new_deck.deal_one())

        print(f"\n***Deal cards***")
        print(f"{player1.name}, you have {player_hands[0]} and {player_hands[1]}. Total: {calculate_blackjack_total(player_hands)} points")
        print(f"Dealer shows 1 of his 2 cards on the table. {dealer_hands[0]}")
        print("\n***Player's Decision***")

        player_decision = None

        while player_decision not in ("hit", "stand"):
            player_decision = input(f"{player1.name}, do you wish to hit or stand? ")

            if player_decision == "hit":
                player_hands.append(new_deck.deal_one())

                player_hands_total = calculate_blackjack_total(player_hands)

                #prepare the list of card objects to be printed
                player_hands_str = ', '.join(str(x) for x in player_hands)
                player_decision = None

                print(f"\n{player1.name}, you now have the following {len(player_hands)} cards on hand: \n{player_hands_str}\nTotal: {player_hands_total} points")

            elif player_decision == "stand":
                player_hands_total = calculate_blackjack_total(player_hands)

                #prepare the list of card objects to be printed
                player_hands_str = ', '.join(str(x) for x in player_hands)
                print(f"\n{player1.name}, you have chosen to stand. You now have the following {len(player_hands)} cards on hand: \n{player_hands_str}\nTotal: {player_hands_total} points")
                break

            else:
                print(f"{player1.name}, please only enter 'hit' or 'stand'.")

        #Dealer's turn

        dealer_hands_total = calculate_blackjack_total(dealer_hands)

        while player_hands_total < 21 and dealer_hands_total <= player_hands_total and dealer_hands_total < 21:
            dealer_hands.append(new_deck.deal_one())
            dealer_hands_total = calculate_blackjack_total(dealer_hands)

        dealer_hands_str = ', '.join(str(x) for x in dealer_hands)

        print("\n***Dealer's Decision***")
        print(f"Dealer has the following {len(dealer_hands)} cards on hand: \n{dealer_hands_str}\nTotal: {dealer_hands_total} points")

        #Tabulating Results

        print("\n***RESULTS***")
        if player_hands_total > 21 or (dealer_hands_total > player_hands_total and dealer_hands_total <= 21):
            print(f"Dealer WON!\n")
            print(f"{player1.name}, you have lost ${player_bet} this round!")
            player1.update_wallet(-player_bet, "lose")
        else:
            print(f"{player1.name} WON!\n")

            #Calculate winnings

            if len(player_hands) == 2 and player_hands_total == 21:
                print(f"{player1.name}, you have won ${player_bet * 2} this round!")
                player1.update_wallet(player_bet*2, "win")
            else:
                print(f"{player1.name}, you have won ${player_bet} this round!")
                player1.update_wallet(player_bet, "win")

        print("\n***Player's Stats***")
        player1.display_stats()
        
        print("\n***Continue game?***")
        #Check if player wants to continue playing
        continue_game = None
    
        while continue_game not in {"y", "Y", "Yes","n", "N", "No"}:
            continue_game = input(f"{player1.name}, do you want to continue playing? (y/n) ")
            no_of_rounds += 1

In [155]:
play_blackjack()


***Start Round 1***
John, please place your bet. You have $200 in your wallet 100
John, you have bet $100.

***Deal cards***
John, you have Four of Clubs and Three of Clubs. Total: 7 points
Dealer shows 1 of his 2 cards on the table. Nine of Diamonds

***Player's Decision***
John, do you wish to hit or stand? hit

John, you now have the following 3 cards on hand: 
Four of Clubs, Three of Clubs, Six of Diamonds
Total: 13 points
John, do you wish to hit or stand? hit

John, you now have the following 4 cards on hand: 
Four of Clubs, Three of Clubs, Six of Diamonds, Four of Spades
Total: 17 points
John, do you wish to hit or stand? hit

John, you now have the following 5 cards on hand: 
Four of Clubs, Three of Clubs, Six of Diamonds, Four of Spades, Three of Spades
Total: 20 points
John, do you wish to hit or stand? stand

John, you have chosen to stand. You now have the following 5 cards on hand: 
Four of Clubs, Three of Clubs, Six of Diamonds, Four of Spades, Three of Spades
Total: 20 